# Retinanet

## Download the Repo
This is the forked version of [the repo](https://github.com/yhenon/pytorch-retinanet) we referenced. We commited some changes to the original code base and added some utilities.

In [ ]:
!git clone -b practice https://github.com/Alan-Kuan/pytorch-retinanet

In [ ]:
%cd pytorch-retinanet

## Train Test Split
$\text{training set} : \text{validation set} : \text{testing set} = 90:7:3$

In [ ]:
annots = os.listdir('/kaggle/input/face-mask-detection/annotations')
from sklearn.model_selection import train_test_split
train, test = train_test_split(annots, test_size=0.1, random_state=22)
test, val = train_test_split(test, test_size=0.7, random_state=22)

## Process Data Annotation
process the data annotation to the repo's format

In [ ]:
!pip install xmltodict
!pip install pycocotools

In [ ]:
import os
import random
import math
import xmltodict
import xml.etree.ElementTree as ET

In [ ]:
annot_path = '/kaggle/input/face-mask-detection/annotations'
img_path = '/kaggle/input/face-mask-detection/images'

for data_type, paths in zip(['train', 'val', 'test'], [train, val, test]):
    with open(f'{ data_type }_annots.csv', 'w') as f:
        for annot_file in paths:
            path = f'{ annot_path }/{ annot_file }'
            tree = ET.parse(path)
            xml_data = tree.getroot()
            xmlstr = ET.tostring(xml_data, encoding='utf-8', method='xml')
            annot = dict(xmltodict.parse(xmlstr))

            img_file = annot['annotation']['filename']
            objs = annot['annotation']['object']
            if not isinstance(objs, list):
                objs = [objs]
            for obj in objs:
                xmin = obj['bndbox']['xmin']
                ymin = obj['bndbox']['ymin']
                xmax = obj['bndbox']['xmax']
                ymax = obj['bndbox']['ymax']
                name = obj['name']
                f.write(f'{ img_path }/{ img_file },{ xmin },{ ymin },{ xmax },{ ymax },{ name }\n')

In [ ]:
with open('./class_list.csv', 'w') as f:
    f.write('without_mask,0\n')
    f.write('with_mask,1\n')
    f.write('mask_weared_incorrect,2\n')

## Training

In [ ]:
!mkdir precision-recall

In [ ]:
!python train.py \
    --dataset csv \
    --csv_train ./train_annots.csv \
    --csv_classes ./class_list.csv \
    --csv_val ./val_annots.csv \
    --graph_save_path ./precision-recall \
    --epoch 200 \
    --model /kaggle/input/retinanetoutput2/pytorch-retinanet/model_final.pt

## Video Input
Feed video input to Retinanet; however, it failed.

In [ ]:
# !python dynamic_prediction.py \
#     --model "/kaggle/input/retinanetoutput2/pytorch-retinanet/model_final.pt" \
#     --video "/kaggle/input/facemaskrecording1/2022-06-08 19-54-17.mkv"

## Speed Test
calculate the average inference time

In [ ]:
!python speed_test.py \
    --model "/kaggle/input/retinanetoutput2/pytorch-retinanet/model_final.pt" \
    --dataset csv \
    --csv_classes ./class_list.csv \
    --csv_val ./test_annots.csv